In [1]:
import os
import numpy as np
import cv2
data_path = r"E:\data_share_ths\dataset\UCF-101\images\test"

In [2]:

image_labels  = {}
images_folder_counts = {}


for folder_name in os.listdir(data_path):
    class_name = folder_name.split("_")[1].lower()
    image_paths = []
    
    for filename in os.listdir(data_path+"\\"+folder_name):
        image_paths.append(data_path+"\\"+folder_name+"\\"+filename)
    images_folder_counts[folder_name] = image_paths
    image_labels[class_name] = image_paths

print(image_labels.keys())
print(len(image_labels['applyeyemakeup']))
print(len(image_labels['applylipstick']))
print(len(image_labels['archery']))



dict_keys(['applyeyemakeup', 'applylipstick', 'archery'])
115
309
158


In [3]:
#images_folder_counts

In [4]:
img_interval = 5
batch_size = 4

In [5]:
### Add label for black image 
real_label = []
real_images =[]
for className,image_paths in images_folder_counts.items():
    temp_img_path = image_paths
    while len(temp_img_path)%img_interval != 0:
        temp_img_path.append("-")
    images_folder_counts[className] = temp_img_path

for key,file_len in images_folder_counts.items():
    print(key,len(file_len))
    

v_ApplyEyeMakeup_g03_c01 210
v_ApplyEyeMakeup_g03_c02 135
v_ApplyEyeMakeup_g03_c03 115
v_ApplyLipstick_g01_c01 150
v_ApplyLipstick_g01_c02 160
v_ApplyLipstick_g01_c03 310
v_Archery_g01_c01 135
v_Archery_g01_c02 145
v_Archery_g01_c03 160


In [59]:
## Create motin pair
X_total,Y_total =[],[]

for folder_name,file_paths in images_folder_counts.items():
    class_name = folder_name.split("_")[1].lower()
    temp_paths = []
    for i in range(len(file_paths)):

        if (i+1)%img_interval !=0:
            temp_paths.append(file_paths[i])
        else:
            Y_total.append(class_name)
            X_total.append(temp_paths)
            temp_paths = []
        

print("X_total :", len(X_total))
print(X_total[0])
print()
print("Y_total :", len(Y_total))
print(Y_total[0])


X_total : 304
['E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000001.jpg', 'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000002.jpg', 'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000003.jpg', 'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000004.jpg']

Y_total : 304
applyeyemakeup


In [60]:
test_amount = 10
limit = len(X_total) - int(len(X_total)/test_amount)

X_train,Y_train =X_total[:limit],Y_total[:limit]
X_test,Y_test =X_total[limit:],Y_total[limit:]

### Custom data generator

In [61]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNEL = 3

BATCH_SIZE = 16
OUTPUT_CHANNELS = 3
EPOCHS = 5

In [62]:
import tensorflow as tf

In [76]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, X_col, Y_col,
                 batch_size,
                 input_size=(224, 224, 3)):
        self.X_col =X_col
        self.Y_col =Y_col
        self.batch_size = batch_size
        self.n = len(self.X_col)

    def read_img(self,path):
        print(path)
        #print("OK na sa",path)
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
        
        #convert to tf and normalize
        tf_img = tf.cast(img, tf.float32)
        tf_img = (tf_img/127.5)-1
        return tf_img
    
    def create_emptyImage(self):
        img= np.zeros((IMG_WIDTH,IMG_HEIGHT,CHANNEL),dtype=int)
        tf_img = tf.cast(img, tf.float32)
        tf_img = (tf_img/127.5)-1
        return tf_img



    def __get_input(self, batch_list):
        batch_img_list = []
        
        for path_lists in batch_list:
            path_img_list = []
            for cur_path in path_lists:
                print(path_lists)
                cur_img = None 
                if cur_path =="-":
                    cur_img = self.create_emptyImage()
                else:
                    cur_img = self.read_img(cur_path)
                path_img_list.append(cur_img)
            path_img_list = np.array(path_img_list)
        batch_img_list.append(path_img_list)
        return batch_img_list
    

    
    def __get_output(self, label):
        return label
    
    def __get_data(self, x_batches, y_batches):
        # Generates data containing batch_size samples
        X_batchs = np.asarray([self.__get_input(x) for x in x_batches])

        y_batches = np.asarray(self.__get_output(y_batches))
        return X_batchs,y_batches
    
    def __getitem__(self, index):
        
        X_batches = self.X_col[index*self.batch_size : (index+1)*self.batch_size]
        Y_batches = self.Y_col[index*self.batch_size : (index+1)*self.batch_size]
        X,y= self.__get_data(X_batches,Y_batches)       
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size
        

In [77]:
test_gen = CustomDataGen(X_col=X_train,Y_col=Y_train,batch_size=batch_size)

In [78]:
for img,label in test_gen:
    print(label)
    print(len(img))
    break

E:\data_share_ths\dataset\UCF-101\images\test\v_ApplyEyeMakeup_g03_c01\frame000001.jpg
E


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
X_train

[['E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000001.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000002.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000003.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000004.jpg'],
 ['E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000006.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000007.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000008.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000009.jpg'],
 ['E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000011.jpg',
  'E:\\data_share_ths\\dataset\\UCF-101\\images\\test\\v_ApplyEyeMakeup_g03_c01\\frame000012.jpg',
  'E:\\d